# Fault diagnostic using simulated data

We have computers and we want to know whether their cpu is broken or not.   
To be able to know that we use the model of the digital twin to create a model of the cpu.   
The digital twin is a way to modelize something material with a program and visualize it on the computer.  
This model allows us to create a lot of data that we will use to train an AI model.

In [ ]:
import random as rd
import pandas as pd

from cosapp.drivers import NonLinearSolver, RungeKutta, LinearDoE, RunSingleCase
from cosapp.recorders import DataFrameRecorder

from cpu.systems import CPUSystem

## Sequence definition of dataset simulation

To create our datasets we used two digital twins of a cpu made up of a fan, a cpu, a heat exchanger and a controler. The first is working without issue, the second has a broken fan and can't cool the cpu.      The simulation is a usage of 100% for 20 seconds and then 0% for 10 seconds. We take the temperature of the cpu after those 30 seconds.   

We then run each digital twins for one thousand air temperature points (with the same distance between two adjacent points) from 0 to 30°C, for a total of two thousand cases. We then choose randomly 200 samples of the broken twin and 800 from the one working.   
To avoid giving the exact same data in the training and testing sets we won't use the same number of points when running the digital twins: for the test set we will make one thousand and one. We then get the two thousand first cases (one thousand broken and the others working) to create our test set.   
The data given in the dataset impact the way our neural network learns. Try to change the dataset: number, pourcentage of each class...   

The data in our datasets are: the temperature of the air, the temperature of the cpu and the tension that should be used to make the fan spin.

In [ ]:
sampleNumbers = 1001
datasetSize = 1000
percentageBroken = 20
rd.seed(9)

In [ ]:
cpu = CPUSystem("cpu")
cpu.fan.mass_flow_scalar = 1.0

design = cpu.add_driver(NonLinearSolver("solver"))
runner = design.add_driver(RunSingleCase("runner"))
design.extend(cpu.design_methods["exchanger_surface"])

design.runner.set_values({"fan.T_air": 40.0, "T_cpu": 80.0, "cpu.usage": 100.0})

# run design
cpu.run_drivers()

In [ ]:
cpu.exchanger.surface

In [ ]:
doe = cpu.add_driver(LinearDoE("doe"))

In [ ]:
time_driver = doe.add_child(RungeKutta(order=3))
solver = time_driver.add_child(NonLinearSolver("solver", max_iter=10, factor=1.0))
time_driver.time_interval = [0, 30]
time_driver.dt = 0.5

# Define a simulation scenario
time_driver.set_scenario(
    init={"T_cpu": 30, "fan.mass_flow_scalar": 1.0},
    values={
        # "fan.T_air": 40., mettre en commentaire !!! sinon remet à zéro quand on appelle time_driver
        "cpu.usage": "100 if time <20 else 0."
    },
)

In [ ]:
doe.add_input_var({"fan.T_air": {"lower": 0.0, "upper": 30.0, "count": sampleNumbers}})

In [ ]:
doe.add_recorder(
    DataFrameRecorder(
        includes=["fan.tension", "cpu.usage", "T_cpu", "fan.T_air", "exchanger.surface"]
    )
)

In [ ]:
cpu.fan.T_air

In [ ]:
cpu.run_drivers()

# Dataset creation

In [ ]:
df = doe.recorder.data

In [ ]:
working = [True for k in range(len(df))]

In [ ]:
df = df.assign(working=working)
df

In [ ]:
time_driver.set_scenario(
    init={"T_cpu": 30, "fan.mass_flow_scalar": 0.0},
    values={
        # "fan.T_air": 40., mettre en commentaire !!! sinon remet à zéro quand on appelle time_driver
        "cpu.usage": "100 if time <20 else 0."
    },
)

In [ ]:
cpu.run_drivers()

# Test set creation

In [ ]:
df2 = doe.recorder.data

In [ ]:
working = [False for k in range(len(df2))]

In [ ]:
df2 = df2.assign(working=working)
df2

In [ ]:
dfclean = df.drop(
    ["Section", "Status", "Error code", "Reference", "cpu.usage", "exchanger.surface"], axis=1
)

In [ ]:
df2clean = df2.drop(
    ["Section", "Status", "Error code", "Reference", "cpu.usage", "exchanger.surface"], axis=1
)

In [ ]:
dataset = []
for k in range(datasetSize):
    if k < datasetSize * percentageBroken / 100:
        i = rd.randint(0, len(df2clean) - 1)
        dataset.append(df2clean.iloc[i])
        df2clean = df2clean.drop(df2clean.index[i])
    else:
        i = rd.randint(0, len(dfclean) - 1)
        dataset.append(dfclean.iloc[i])
        dfclean = dfclean.drop(dfclean.index[i])
    dataset[k].name = k

dfclean = df.drop(
    ["Section", "Status", "Error code", "Reference", "cpu.usage", "exchanger.surface"], axis=1
)
df2clean = df2.drop(
    ["Section", "Status", "Error code", "Reference", "cpu.usage", "exchanger.surface"], axis=1
)

In [ ]:
"""
dataset=[]
for k in range(tailleDataset):
    if k < tailleDataset*percentageCasse/100:
        i=rd.randint(0, len(dfclean)-1)
        dataset.append(df2clean.iloc[i])
    else:
        i=rd.randint(0, len(dfclean)-1)
        dataset.append(dfclean.iloc[i])
    dataset[k].name=k
"""

In [ ]:
type(dfclean.iloc[1])

In [ ]:
cols = ["T_cpu", "fan.T_air", "fan.tension", "working"]
dfFinal = pd.DataFrame(dataset, columns=cols)

In [ ]:
dfFinal

In [ ]:
dfFinal.to_csv(
    f"data/dataset_{datasetSize}_cases_{percentageBroken}_percent_broken.csv", index=False
)

In [ ]:
dataset = []
for k in range(datasetSize):
    dataset.append(df2clean.iloc[k])
    dataset.append(dfclean.iloc[k])
    dataset[2 * k].name = 2 * k
    dataset[2 * k + 1].name = 2 * k + 1

In [ ]:
len(dataset)

In [ ]:
cols = ["T_cpu", "fan.T_air", "fan.tension", "working"]
testfinal = pd.DataFrame(dataset, columns=cols)
testfinal.to_csv(
    f"data/test_set_{datasetSize}_cases_{percentageBroken}_percent_broken.csv", index=False
)